## Python Room N°3

**Collaboratore: Lucchesi Simone Luca**


<img src="../Images/SFClub_Siena.png" width="150">

# **INDICE**


[**1**)](#1) **Introduzione.**
   
   
[**2)**](#2) **Applicazione N°1: Regressione lineare semplice in poche righe.** 


   - [**2.1)**](#2.1) Utilizzo dati settimanali scaricati da Investing.com.
    

   - [**2.2)**](#2.2) Regressione lineare con il metodo dei minimi quadrati (OLS).


[**3)**](#3) **Apllicazione N°2: Esempio di simulazione Monte Carlo e applicazione del teorema del limite centrale.**


   - [**3.1)**](#3.1) PRNG: Pseudo Random Number Generator. 
    

   - [**3.2)**](#3.2) Applicazione di una Simulazione Montecarlo: lanciare in successione due monete ed ottenere sempre testa.
   
        - [**3.2.1)**](#3.2.1) Convergenza dal punto di vista grafico. 
      
      
- [**3.3)**](#3.3) Accuratezza di una Simulazione Montecarlo: Teorema del limite centrale. 
  

# 1) Introduzione <a name ="1"></a>

# 2) Applicazione N°1: Regressione lineare semplice in poche righe. <a name="2"></a>

Vediamo come: importare, analizzare ed interpretare (graficamente e statisticamente) un paio di serie temporali: l'andamento del prezzo del FTSEMIB nel periodo 01/01/2019 e 01/01/2020 di contrasto all'andamento del titolo azionario A2A per lo stesso intervallo temporale. 

_Per semplicità non verranno definiti molti passaggi, assunti per il momento come dati._ 

In [ ]:
#Import preliminare 
import numpy as np 
import pandas as pd 
import os #Modulo utile per la gestione di file 
import matplotlib.pyplot as plt 
from scipy import stats #Modulo per la media geometrica (e non solo!)

La seguente è una _funzione di utilità_ da me creata in passato per la gestione dei dataset scaricati gratuitamente da Investing in formato .csv (comma separated variables). 

In [ ]:
#Utility function

def removeFile(fileName):
    """
    removeFile(fileName) function remove file 'fileName', if it exists. It also prints on screen a success/failure message.
    
    Parameters:
        fileName (str): name of the file ('Data' folder is assumed)
        
    Returns:
        None
    """

    if os.path.isfile(os.path.join(dataPath, fileName)):
        os.remove(os.path.join(dataPath, fileName))

        # double-check if file still exists
        fileStillExists = os.path.isfile(os.path.join(dataPath, fileName))

        if fileStillExists:
            print("Failure: file {} still exists...".format(fileName))
        else:
            print("Success: file {} successfully removed!".format(fileName))
            
    else:
        print("File {} already removed.".format(fileName))

In [ ]:
def modInvestingComplete(dataFrame): 
   
    import datetime as dt # It can be omitted if preliminary imported
    
    """
    This function  starts modifying the DataFrame index composed by strings starting from element's aspetct 
    (converting dots to minus) and then modifying their order into convention "Y-M-D". Finally, it converts the list into a 
    dateTime object wich will replace the previous index. 
    All in order to have a correct index composed by dates which can be passed to get every useful result. 
     
    
    Then it passes to dataFrame elements removing points and converting commas into points for every dataframe column.
    Thereafter it converts strings to float data type for each element in columns. 
    This is a correction needed to import dataframes from Investing csv's. 
    This is a leaner alternative to previous function, more compact but equally efficient.
    
    modInvesting(dataFrame) function first of all converts index into a list data type,  respectively "." to " " and "," to "." in dataFrames according to American convention. 
    Then it transform each value in each column from strings to float data type. 
    
    Parameter: 
        (dataFrame): dataFrame data structure I imported from Investing.com, each element is a string. 
         
    Results:
        (dataFrame): the same data frame with values converted to make data analysis.  
    """




    strDates = dataFrame.index.tolist()
    
    for i in range(len(strDates)):
        strDates[i] = strDates[i].replace('.','-')
            
    dates = [dt.datetime.strptime(strDates[i], "%d-%m-%Y") for i in range(len(strDates))]
    
    newStrDates = [dt.datetime.strftime(dates[i], "%Y-%m-%d") for i in range (len(dates))]
    
    newDates = [dt.datetime.strptime(newStrDates[i], "%Y-%m-%d") for i in range(len(newStrDates))]

    dataFrame.index = newDates
    
    
    
#With the following command I apply the function lambda to replace points with white bars to all elements of each column
    
    dataFrame = dataFrame.apply(lambda x: x.str.replace(".", ""))
    
#With the following command I apply the function lambda to replace commas with points to all elements of each column
    
    dataFrame = dataFrame.apply(lambda x: x.str.replace(",", "."))
    
#With the following command I apply the function lambda to convert in a numeric format all strings values in each column
    
    dataFrame = dataFrame.apply(pd.to_numeric)
    
#Finally, I can revert the order of rows in dataFrame object
    
    dataFrame = dataFrame.iloc[::-1,::]
    
    
    
    return  dataFrame

## 2.1) Utilizzo dati settimanali scaricati da Investing.com <a name="2.1"></a>

**Utilizzo dati settimanali per A2A da Investing.com**

Una volta scaricati tali dati, in formato .CSV possiamo importarli in Python utilizzando l'apposita funzione presente nel paccheto Pandas. 


In [ ]:
dataPath = "./Dataset"
filePath = os.path.join(dataPath, "A2A wk 10y dirty.csv")

In [ ]:
a2aWk = pd.read_csv(filePath, index_col=0)

In [ ]:
a2aWk

Facciamo un pò di pulizia rimuovendo due colonne da questa struttura di tipo pd.DataFrame

In [ ]:
del a2aWk["Vol."]

In [ ]:
del a2aWk["Var. %"]

In [ ]:
a2aWk

Applichiamo l'utility function prima definita per avere una time series più ordinata focalizzata esclusivamente sul prezzo di chiusura settimanale (Ultimo).

In [ ]:
adjA2aWkClose = modInvestingComplete(a2aWk).loc["2010-01-02":"2020-01-02", "Ultimo"]

In [ ]:
adjA2aWkClose

Abbiamo adesso una serie temporale ordinata dei prezzi di chiusura settimanali nel periodo "2010-01-02 / 2020-01-02"

Sfruttando la seguente procedura andiamo a creare una struttura simile, che semplicemente sposta in avanti di un frame (1 settimana) l'attuale raccolta dati. 

In [ ]:
a2aClosePrice_t1 = adjA2aWkClose
a2aClosePrice_tm1 = a2aClosePrice_t1.shift(periods=1, freq = "1w")

In [ ]:
a2aClosePrice_tm1

operazione) possiamo applicare la seguente formula per calcolare il _tasso di rendimento logaritmico settimanale_ (assumendo un regime di capitalizzazione esponenziale): 

log(P_t+1/P_t) = log(P_t+1) - log(P_t) ) r_t,t+1 

In [ ]:
a2aWkLogRet = np.log(a2aClosePrice_t1) - np.log(a2aClosePrice_tm1)
a2aWkLogRet = a2aWkLogRet.dropna() 

In [ ]:
a2aWkLogRet

Ovviamente un'osservazione, nel calcolo del rendimento, viene persa. 

In [ ]:
a2aWkLogRet.describe()

Per quanto riguarda la media geometrica del rendimento settimanale: 

In [ ]:
avgWkRate = stats.gmean(a2aWkLogRet +1 ) -1
print(avgWkRate)

**Grafico delle oscillazioni di rendimento in confronto al rendimento medio settimanale nel periodo di osservazione**

In [ ]:
avgWkRateSeries= pd.Series(data = [avgWkRate for i in range(a2aWkLogRet.size)],
                                         index =a2aWkLogRet.index )

plt.figure(figsize=(10,6))
plt.plot(a2aWkLogRet,'b',lw=1.5, label ='WKLogRet')
plt.plot(avgWkRateSeries, 'r', lw =1.5, label ='AverageRet')
plt.legend (loc=0)
plt.xlabel('Weeks')
plt.ylabel('logRet')
plt.title('Comparison between weekly log returns on Dax and their average')

plt.show()

In [ ]:
removeFile(filePath)

**Utilizzo dati settimanali per FTSE MIB da Investing.com**

Adesso Ripetiamo lo stesso per i dati relativi alle quotazioni dell'indice FTSE MIB40 nello stesso range temporale:

In [ ]:
filePath = "./Dataset"
filePath = os.path.join(dataPath, "Ftsemib  wk 10y dirty.csv")

In [ ]:
ftsemibWk = pd.read_csv(filePath, index_col=0)

In [ ]:
del ftsemibWk["Vol."]

In [ ]:
del ftsemibWk["Var. %"]

In [ ]:
adjFtseMibWkClose = modInvestingComplete(ftsemibWk).loc["2010-01-02":"2020-01-02", "Ultimo"]

In [ ]:
adjFtseMibWkClose

Di stessa lunghezza rispetto al precedente dataset.
Ripetiamo adesso i medesimi passaggi per arrivare alla serie di rendimenti logaritmici settimanali. 

In [ ]:
ftseWkClose_t = adjFtseMibWkClose
ftseWkClose_tm1 = ftseWkClose_t.shift(periods = 1, freq = '1w')

In [ ]:
ftseWkLogRet = np.log(ftseWkClose_t) -  np.log(ftseWkClose_tm1)
ftseWkLogRet = ftseWkLogRet.dropna()

In [ ]:
ftseWkLogRet

## Regressione lineare con il metodo dei minimi quadrati (OLS). <a name="2.2"></a>

Adesso che disponiamo di una soddisfacente serie temporale in termini di rendimenti logaritmici rispettivamente per FTSE MIB40 e il titolo A2A (osservazioni che coincidono in termini di date di osservazione) possiamo disegnare un grafico a dispersione per vedere se esiste o meno una qualche correlazione. 

**Scatter plot (Grafico a dispersione)**

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(ftseWkLogRet, a2aWkLogRet, marker ='o' )
plt.show()

Rispettivamente abbiamo osservazioni del titolo A2A sull'asse Y mentre osservazioni dell'indice FTSEMIB sull'asse delle X. 
Sembra esistere una relazione positiva tra le due variabili.

Per avere più chiarezza è possibile performare una regressione lineare semprice con il metodo dei minimi quadrati (OLS) per individuare i parametri caratterizzanti rispettivamente l'intercetta ed il coefficiente angolare. **Non scenderò nei dettagli in quanto esulano dallo scopo**.

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [ ]:
x = np.array(ftseWkLogRet).reshape((-1,1))
y = np.array(a2aWkLogRet)

In [ ]:
model.fit(x,y)

In [ ]:
y_pred = model.predict(x)

**Risultati**

**1) Intercetta (termine noto)** 

In [ ]:
model.intercept_

**2) Coefficiente angolare (Beta di un CAPM)**

In [ ]:
model.coef_[0]

**3) Rappresentazione grafica della retta di regressione** 

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(ftseWkLogRet, a2aWkLogRet, marker ='o' )
plt.plot(x,y_pred, 'r', lw= 2)

plt.show()

---

# 3) Applicazione N°2: esempio di simulazione Monte Carlo e applicazione del teorema del limite centrale. 

**[NOTA]** Questo contenuto non è ancora stato sviluppato nel dettaglio.

Import preliminare: 

In [ ]:
import random 
import matplotlib.pyplot as plt

## 3.1) PRNG: Pseudo Random Number Generator <a name="3.1"></a>

In [ ]:
k = int(input("Number of iterations: "))

In [ ]:
lis_1 = []
lis_2 = []

lis_check = []

In [ ]:
random.seed(1)

for _ in range(k): 
    
    lis_1.append(random.randint(0,5))


In [ ]:
random.seed(2)

for _ in range(k): 
    
    lis_2.append(random.randint(0,5))

In [ ]:
#Check per PRNG: 

random.seed(1)
for _ in range(k): 
    
    lis_check.append(random.randint(0,5))

In [ ]:
print(lis_1[:5])
print(lis_2[:5])
print(lis_check[:5])

In [ ]:
plt.hist(lis_1,6,range = [-0.5,5.5]) #6 = numero di bins, #range =... il range di riferimento del mio istogramma

plt.show()

## 3.2) Applicazione di una Simulazione Montecarlo: lanciare in successione due monete ed ottenere sempre testa. <a name ="3.2"></a>

Secondo l'approccio frequentista, in cui la probabilità di un evento è data dal rapporto _eventi favorevoli / eventi totali_ possiamo facilmente intuire che la probabilità di ottenere testa o croce sarà rispettivamente data da: 

$\frac{Testa}{(Testa,Croce)}$ $=$ $\frac{1}{2}$ 

Conseguentemente, se decidiamo di voler calcolare la probabilità di ottenere per due volte di seguito testa da due lanci (indipendenti), avremo che: 

$P_{testax2}$ $=$ $\frac{1}{2}$ $.$ $\frac{1}{2}$ $=$ $\frac{1}{4}$ = $0.25$ = $25$%

Proviamo adesso ad implementare una simulazione, verificando attraverso una serie di tentativi (sempre più crescente) la convergenza dei nostri successi a questo livello teorico.

In [ ]:
from random import randint, seed 

- **1)** Definizione del dominio di possibili eventi:

In [ ]:
seed(1)

eventi = ["Testa","Croce"]

eventi

- **2)** Generazione di input randomicamente da una distribuzione di probabilità (uniforme) sul dominio di eventi definito:

In [ ]:
tentativi = 10
successi = 0 #Pre-allocazione della serie di successi che otterremo

for _ in range(tentativi): 
    
    combo = []
    
    lancio_1 = eventi[random.randint(0,1)]
    combo.append(lancio_1)
    
    lancio_2 = eventi[random.randint(0,1)]
    combo.append(lancio_2)

    
    
    
    #print(combo)

    
#Verifica del numero di successi inerenti all'evento composto cui siamo interessati 

    if combo == ["Testa","Testa"]: 
        
        successi +=1 
        
print(successi)
        

- **3)** Aggregazione dei risultati ottenuti: 

In [ ]:
print("La probabilità di ottenere due volte testa è {:.2f}".format(successi/tentativi))

Chiaramente, per un piccolo numero di tentativi è evidente l'assenza di convergenza a livello teorico. 

Basandoci su un approccio frequentista, il nostro risultato potrà avere un riscontro coerente con l'aspettativa teorica solo in seguito ad un grande numero di simulazioni (iterazioni). 

Mettendo insieme quanto prima rappresentato, creiamo una funzione di utilità: 

In [ ]:
def lancio_moneta(lanci = 2, tentativi = 10, richiesta = None):
    
    
    eventi = ["Testa", "Croce"]
    
    successi_testa = 0 
    successi_croce = 0
    
    insuccesso = 0 
    
    for _ in range(tentativi): 
        
        combo = []
        
        for toss in range(lanci): 
            
            lancio = eventi[randint(0,1)]
            combo.append(lancio)
        
        if combo == ["Testa", "Testa"]: 
            
            successi_testa +=1 
        
        elif combo == ["Croce", "Croce"]: 
            
            successi_croce +=1 
            
        else: 
            insuccesso = 0 
            
        
    probabilità_serie_testa = successi_testa/tentativi
    probabilità_serie_croce = successi_croce/tentativi
    probabilità_no_serie = insuccesso/tentativi 
    
    if richiesta == "T": 
        
        return probabilità_serie_testa
    
    elif richiesta =="C": 
        
        return probabilità_serie_croce

    else: 
        return probabilità_no_serie 
    

In [ ]:
prob_serie_testa = lancio_moneta(2,10,"T")
prob_serie_testa 

### 3.2.1) Convergenza da un punto di vista grafico <a name="3.2.1"></a>

Sull'asse delle ascisse del nostro grafico cartesiano abbiamo il numero di simulazioni $N$, mentre come variabile dipendente la probabilità di successo $P(N)$. 

In [ ]:
lista_tentativi=  [i for i in range(1000,100000+1000,1000)]

#print(lista_tentativi)


lista_probabilità = []

for tentativi in lista_tentativi: 
    
    lista_probabilità.append(lancio_moneta(tentativi = tentativi, richiesta = "T"))
    
#print(lista_probabilità)

In [ ]:
print(lista_probabilità[0:5]) #Primi 5 valori della lista, partendo da 1000 tentativi a 5000

In [ ]:
print(lista_probabilità[-5:]) #Ultimi 5 valori della lista

In [ ]:
x = lista_tentativi
y = lista_probabilità 


fig = plt.figure(figsize = (8,5))

plt.plot(x,y, marker = "+", color = "green", label = "Linea di convergenza")
plt.title("Sviluppo della convergenza all'aumentare delle simulazioni dell'esperimento: 2 volte testa")
plt.xlabel("Numero simulazioni")
plt.ylabel("Probabilità di ottenere due volte testa")

plt.hlines(0.25,0,100000,color = "blue", label ="Probabilità teorica") 

plt.legend()
plt.show()

## 3.3) Accuratezza di una Simulazione Montecarlo: Teorema del limite centrale <a name="3.3"><a>

Questo strumento è molto utilizzato in campo di analisi stocastica, basti però pensare che, per semplici applicazioni come quella rappresentata sopra può essere molto _time consuming_ : infatti, per ottenere un risultato facilmente dimostrabile a livello teorico è necessario utilizzare un elevato numero di simulazioni, fattispecie che richiede alla nostra macchina tempo.

Detto questo, tipicamente per poter testare l'accuratezza del nostro metodo facciamo ricorso ad un famoso teorema, molto utilizzato in campo statistico (inferenziale): il **teorema del limite centrale**. 

Senza scendere in troppi dettagli il famoso teorema enuncia che: la media di un grande numero di variabili aleatorie indipendenti e dotate della stessa distribuzione è approssimativamente normale, indipendentemente dalla distribuzione soggiacente. 

Nel nostro caso, la variabile di riferimento risulta essere rappresentata dal tentativo mentre il campione esattamente dal numero di simulazioni. 

Questo implica che la distribuzione di alcune statistiche, per esempio la media del campione, diventa nota anche se non sappiamo nulla a proposito della forma della distribuzione della popolazione da cui i campioni sono stati  estratti. 

Il termine grandezza del campione (numero simulazioni) è relativo, tanto sarà più grande tanto il risultato più accurato. 

Vediamo come la distribuzione, nel nostro caso del rapporto successi/tentativi al variare del numero di esperimenti (ogni esperimento prevederà di simulare N volte il doppio lancio di moneta) si comporta. 

Per ripetere esperimenti diversi utilizzeremo _seed_ diversi. 

In [ ]:
import numpy as np 

esperimento = 1000
simulazioni = 1000

prob_list = []

for e in range(esperimento): 
    
    seed(e)
    
    prob_list.append(lancio_moneta(tentativi = simulazioni, richiesta = "T"))


print("La media di tutti i risultati degli esperimenti è: {}".format(np.mean(prob_list)))    
    
plt.hist(prob_list,50, color = "lightblue", label = "pdf")

plt.vlines(np.mean(prob_list),0,0.08*esperimento, color = "red",label = "Mean")

plt.grid()
plt.legend()
plt.show()

**- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -** 

Il resto del contenuto verrà trattato nei successivi notebook. 

Al momento la produzione è la seguente: 

**1)** [Python Room](https://github.com/simonelucchesi/Python_room/blob/main/Notebooks/Python%20Room%20.ipynb) 

**2)** [Python Room 2](https://github.com/simonelucchesi/Python_room/blob/main/Notebooks/Python%20room%202%20.ipynb)

**3)** Python Room 3 

